In [1]:
import pandas as pd

from google_play_scraper import app, Sort, reviews

from pprint import pprint

import pymongo
from pymongo import MongoClient

import datetime as dt
from tzlocal import get_localzone

import random
import time

import os

In [4]:
client = MongoClient(host='localhost', port=27017)

play_store_db = client['play_store_db']

info_collection = play_store_db['info_collection']

comment_collection = play_store_db['comment_collection']

In [5]:
# app_df = pd.read_csv('Var/PlayStore/ScrapApp.csv.csv')
# app_df.head()
app_name = []
android_appID = []

In [6]:
# number of apps as input
n = int(input("Enter number of apps : "))

# iterating till the range
for i in range(0, n):
    appName = input("Enter App Name: ")
    appID = input("Enter App ID: ")
    # adding the appName and appID
    app_name.append(appName) 
    android_appID.append(appID)

print(app_name)
print(android_appID)

['SpongeBob Adventures: In A Jam', 'SpongeBob: Cooking Fever']
['com.tiltingpoint.sbadventures', 'com.tiltingpoint.spongebob']


In [7]:
app_names = list(app_name)
app_ids = list(android_appID)

In [8]:
app_info = []
for i in app_ids:
    info = app(i)
    del info['comments']
    app_info.append(info)

pprint(app_info[0])

{'adSupported': True,
 'appId': 'com.tiltingpoint.sbadventures',
 'categories': [{'id': 'GAME_SIMULATION', 'name': 'Simulation'},
                {'id': None, 'name': 'Management'},
                {'id': None, 'name': 'Farming'},
                {'id': 'GAME_CASUAL', 'name': 'Casual'},
                {'id': None, 'name': 'Single player'},
                {'id': None, 'name': 'Stylized'},
                {'id': None, 'name': 'Cartoon'},
                {'id': 'GAME_SIMULATION', 'name': 'Simulation'},
                {'id': None, 'name': 'Time management'},
                {'id': 'GAME_ARCADE', 'name': 'Arcade'},
                {'id': None, 'name': 'Single player'},
                {'id': None, 'name': 'Stylized'},
                {'id': None, 'name': 'Cartoon'},
                {'id': None, 'name': 'Offline'}],
 'containsAds': True,
 'contentRating': 'Everyone',
 'contentRatingDescription': None,
 'currency': 'USD',
 'description': 'Get Ready to go on an Amazing Adventure with Sponge

In [9]:
info_collection.insert_many(app_info)

In [10]:
info_df = pd.DataFrame(list(info_collection.find({})))
info_df.head()

,_id,title,description,descriptionHTML,summary,installs,minInstalls,realInstalls,score,ratings,...,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,appId,url
0,653b319d96112b2154a38f9d,SpongeBob Adventures: In A Jam,Get Ready to go on an Amazing Adventure with S...,Get Ready to go on an Amazing Adventure with S...,"Build, Adventure and Explore in a Wondrous Wor...","1,000,000+",1000000,3468376,4.486865,26850,...,https://play-lh.googleusercontent.com/2-PF9SvG...,Everyone,None,True,True,"Sep 26, 2023",1697249191,2.1.0,com.tiltingpoint.sbadventures,https://play.google.com/store/apps/details?id=...
1,653b319d96112b2154a38f9e,SpongeBob: Krusty Cook-Off,Cook delicious food and drinks in this FREE on...,Cook delicious food and drinks in this FREE on...,Play with SpongeBob in this cooking challenge ...,"50,000,000+",50000000,58768077,4.331146,373770,...,https://play-lh.googleusercontent.com/9KYQnVNZ...,Everyone,None,True,True,"May 12, 2020",1691598725,5.4.4,com.tiltingpoint.spongebob,https://play.google.com/store/apps/details?id=...


In [12]:
for app_name, app_id in zip(app_names, app_ids):
    
    start = dt.datetime.now(tz=get_localzone())
    fmt= "%m/%d/%y - %T %p"    
    
    print('---'*20)
    print('---'*20)    
    print(f'***** {app_name} started at {start.strftime(fmt)}')
    print()
    
    app_reviews = []
    
    count = 200
    
    batch_num = 0
    
    # Retrieve reviews (and continuation_token) with reviews function
    rvws, token = reviews(
        app_id,           # found in app's url
        lang='en',        # defaults to 'en'
        country='us',     # defaults to 'us'
        sort=Sort.NEWEST, # start with most recent
        count=count       # batch size
    )  # type: ignore
    
    for r in rvws:
        r['app_name'] = app_name # add key for app's name
        r['app_id'] = app_id     # add key for app's id
    
    
    app_reviews.extend(rvws)
    
    batch_num +=1 
    print(f'Batch {batch_num} completed.')
    
    time.sleep(random.randint(1,5))
    
    pre_review_ids = []
    for rvw in app_reviews:
        pre_review_ids.append(rvw['reviewId'])
    
    for batch in range(4999):
        rvws, token = reviews( # store continuation_token
            app_id,
            lang='en',
            country='us',
            sort=Sort.NEWEST,
            count=count,
            # using token obtained from previous batch
            continuation_token=token
        )
        
        new_review_ids = []
        for r in rvws:
            new_review_ids.append(r['reviewId'])
            
            r['app_name'] = app_name # add key for app's name
            r['app_id'] = app_id     # add key for app's id
    
        app_reviews.extend(rvws)
        
        batch_num +=1
        
        # Break loop and stop scraping for current app if most recent batch
        # did not add any unique reviews
        all_review_ids = pre_review_ids + new_review_ids
        if len(set(pre_review_ids)) == len(set(all_review_ids)):
            print(f'No reviews left to scrape. Completed {batch_num} batches.\n')
            break
        
        # all_review_ids becomes pre_review_ids to check against 
        # for next batch
        pre_review_ids = all_review_ids
        
        # At every 100th batch
        if batch_num%100==0:
            
            # print update on number of batches
            print(f'Batch {batch_num} completed.')
            
            # insert reviews into collection
            comment_collection.insert_many(app_reviews)
            
            output_path = 'results/' + app_name + '.csv' 
            review_df = pd.DataFrame(app_reviews)
            review_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
            
            # print update about num reviews inserted
            store_time = dt.datetime.now(tz=get_localzone())
            print(f"""
            Successfully inserted {len(app_reviews)} {app_name} 
            reviews into collection at {store_time.strftime(fmt)}.\n
            """)
            
            # empty our list for next round of 100 batches
            app_reviews = []
        
        time.sleep(random.randint(1,5))
    
    # Print update when max number of batches has been reached
    # OR when last batch didn't add any unique reviews
    print(f'Done scraping {app_name}.')
    print(f'Scraped a total of {len(set(pre_review_ids))} unique reviews.\n')
    
    
    # Insert remaining reviews into collection
    comment_collection.insert_many(app_reviews)
    
    output_path = 'results/' + app_name + '.csv' 
    review_df = pd.DataFrame(app_reviews)
    review_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    
    # Get end time
    end = dt.datetime.now(tz=get_localzone())
    
    # Print ending output for app
    print(f"""
    Successfully inserted all {app_name} reviews into collection
    at {end.strftime(fmt)}.\n
    """)
    print(f'Time elapsed for {app_name}: {end-start}')
    print('---'*20)
    print('---'*20)
    print('\n')
    
    time.sleep(random.randint(1,5))

------------------------------------------------------------
------------------------------------------------------------
***** SpongeBob Adventures: In A Jam started at 10/27/23 - 10:44:53 AM

Batch 1 completed.
No reviews left to scrape. Completed 11 batches.

Done scraping SpongeBob Adventures: In A Jam.
Scraped a total of 1946 unique reviews.


    Successfully inserted all SpongeBob Adventures: In A Jam reviews into collection
    at 10/27/23 - 10:45:27 AM.

    
Time elapsed for SpongeBob Adventures: In A Jam: 0:00:34.327394
------------------------------------------------------------
------------------------------------------------------------


------------------------------------------------------------
------------------------------------------------------------
***** SpongeBob: Cooking Fever started at 10/27/23 - 10:45:28 AM

Batch 1 completed.
Batch 100 completed.

            Successfully inserted 20000 SpongeBob: Cooking Fever 
            reviews into collection at 10/27